<a href="https://colab.research.google.com/github/Edward-TL/amazon_scraper/blob/master/amazon_scraper_boxes_df.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [19]:
%cd "drive"
%cd 'My Drive'
%cd 'Colab Notebooks'
%cd '4SS/4SS_db/testing'
%mkdir 'mx'
%mkdir 'br'

[Errno 2] No such file or directory: 'drive'
/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing
[Errno 2] No such file or directory: 'My Drive'
/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing
[Errno 2] No such file or directory: 'Colab Notebooks'
/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing
[Errno 2] No such file or directory: '4SS/4SS_db/testing'
/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing


In [29]:
from bs4 import BeautifulSoup
from openpyxl.workbook import Workbook

import os
from datetime import datetime

import requests

import pandas as pd

def extract_soup(url, preview=True):
    response = requests.get(url)
    print(f'Request status: {response.status_code}\n')

    soup = BeautifulSoup(response.text, 'lxml')

    if preview==True:
        print(soup.prettify())

    return soup

def top_amazon_boxes(soup):
    boxes = soup.find_all('div', attrs={'class':"a-section a-spacing-none aok-relative"})

    return boxes

def scrape_boxes(boxes, domain):
    
    ranks = [None]*50
    product_names = [None]*50
    image_urls = [None]*50
    product_links = [None]*50
    star_ratings = [None]*50
    reviews = [None]*50
    authors_companies = [None]*50
    editions_consoles = [None]*50
    min_prices = [None]*50
    max_prices = [None]*50

    amz_mx_url = 'https://www.amazon.com.mx'
    
    n_box = 0
    for box in boxes:
        rank_box = box.find_all('span', attrs={'class':'zg-badge-text'})
        products_and_image_box = box.find_all('div', attrs={'class' : 'a-section a-spacing-small'})
        product_links_box = box.find_all('a', attrs={'class' : 'a-link-normal'})
        star_ratings_box = box.find_all('span', attrs={'class' : 'a-icon-alt'})
        reviews_box = box.find_all('a', attrs={'class' : 'a-size-small a-link-normal'})
        authors_company_box = box.find_all('span', attrs={'class' : 'a-size-small a-color-base'})
        editions_console_box = box.find_all('span', attrs={'class' : 'a-size-small a-color-secondary'})
        prices_box = box.find_all('span', attrs={'class' : "p13n-sc-price"})
        
        
        ranks[n_box] = rank_box[0].get_text()
        product_names[n_box] = products_and_image_box[0].img.get('alt')
        image_urls[n_box] = products_and_image_box[0].img.get('src')
        product_links[n_box] = amz_mx_url + product_links_box[0].get('href')

        #Depended cases
        try:
            star_ratings[n_box] = float(star_ratings_box[0].get_text()[:3])
            reviews[n_box] = int(reviews_box[0].get_text().replace(',',''))
        except:
            star_ratings[n_box] = None
            reviews[n_box] = None

        #Individual cases
        try:
            authors_companies[n_box] = authors_company_box[0].get_text()
        except:
            authors_companies[n_box] = None
        
        try:
            editions_consoles[n_box] = editions_console_box[0].get_text()
        except:
            editions_consoles[n_box] = None

        if domain == 'mx':
            coin_symbol = 1
        elif domain == 'br':
            coin_symbol = 2

        try:
            min_prices[n_box] = float(prices_box[0].get_text()[coin_symbol:].replace(',',''))
        except:
            min_prices[n_box] = None

        try:    
            max_prices[n_box] = float(prices_box[1].get_text()[coin_symbol:].replace(',',''))
        except:
            max_prices[n_box] = None

        n_box = n_box + 1 


    # Dictionary
    boxes_dict = {
    "Rank" : ranks,
    "Product Names": product_names,
    "Image urls": image_urls,
    "Product links": product_links,
    "Stars": star_ratings,
    "Reviews": reviews,
    "Authors/Company": authors_companies,
    "Edition/Console": editions_consoles,
    "Price_std_or_min" : min_prices,
    "Max_prices" : max_prices
    }


    return boxes_dict

In [24]:
URL_dict = {
    'top_food_and_drinks' : 'grocery',
    'top_automotriz' : 'automotive',
    'top_amazon-devices' : 'amazon-devices',
    'top_bebe': 'baby',
    'top_dep_al' : 'sports',
    'top_electronicos' : 'electronics',
    'top_herramientas' : 'tools',
    'top_hogar_cocina': 'kitchen',
    'top_industrial_emp_ciencia' : 'industrial',
    'top_musical_instruments' : 'musical-instruments',
    'top_juguetes_juegos' : 'toys',
    'top_libros':'books',
    'top_musica' : 'music',
    'top_oficina_papeleria' : 'officeproduct',
    'top_peliculas_series' : 'dvd',
    'top_handmade' : 'handmade',
    'top_prod_animales' : 'pet-supplies',
    'top_ropa_zapatos_acc' : 'shoes',
    'top_salud' : 'hpc',
    'top_software' : 'software',
    'top_kindle' : 'digital-text',
    'top_videojuegos': 'videogames'
    }

In [20]:
domain_dict = {'Mexico' : 'mx',
               'Brazil' : 'br'}

In [33]:
for country in domain_dict:
    for key in URL_dict:
        
        url = f'https://www.amazon.com.{domain_dict[country]}/gp/bestsellers/{URL_dict[key]}/ref=zg_bs_nav_0'
        print(f'Scraping {URL_dict[key]}')
        print(f'URL given: {url}')

        soup = extract_soup(url, preview=False)
        boxes = top_amazon_boxes(soup)

        amz_key_top_boxes = scrape_boxes(boxes, domain_dict[country])
        amz_key_top_boxes_df = pd.DataFrame.from_dict(amz_key_top_boxes)

        test_time = datetime.now()
        test_time = test_time.strftime('%Y-%m-%d %H:%M')

        csv_file = f'/{domain_dict[country]}-test_{URL_dict[key]}_{test_time}.csv'
        dir_csv_testing = f'/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing/{domain_dict[country]}/{csv_file}'

        #Creating csv files
        amz_key_top_boxes_df.to_csv(dir_csv_testing, sep = '|', index = False)

        excel_file = f'/{domain_dict[country]}-test_{URL_dict[key]}_{test_time}.xlsx'
        dir_excel_testing = f'/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing/{domain_dict[country]}/{excel_file}'
        
        #Creating xlsx files
        amz_key_top_boxes_df.to_excel(dir_excel_testing, index = False)

        print(f'Loaded {domain_dict[country]}-{URL_dict[key]}')
        time.sleep(0.05)



Scraping grocery
URL given: https://www.amazon.com.mx/gp/bestsellers/grocery/ref=zg_bs_nav_0
Request status: 200

Loaded mx-grocery
Scraping automotive
URL given: https://www.amazon.com.mx/gp/bestsellers/automotive/ref=zg_bs_nav_0
Request status: 200

Loaded mx-automotive
Scraping amazon-devices
URL given: https://www.amazon.com.mx/gp/bestsellers/amazon-devices/ref=zg_bs_nav_0
Request status: 200

Loaded mx-amazon-devices
Scraping baby
URL given: https://www.amazon.com.mx/gp/bestsellers/baby/ref=zg_bs_nav_0
Request status: 200

Loaded mx-baby
Scraping sports
URL given: https://www.amazon.com.mx/gp/bestsellers/sports/ref=zg_bs_nav_0
Request status: 200

Loaded mx-sports
Scraping electronics
URL given: https://www.amazon.com.mx/gp/bestsellers/electronics/ref=zg_bs_nav_0
Request status: 200

Loaded mx-electronics
Scraping tools
URL given: https://www.amazon.com.mx/gp/bestsellers/tools/ref=zg_bs_nav_0
Request status: 200

Loaded mx-tools
Scraping kitchen
URL given: https://www.amazon.com.m